# Data Preparation Ticket Resolving for RAG


In this section, we prepare our data for the RAG model by processing PDF documents, dividing the content into manageable chunks, creating embeddings, and storing them for efficient retrieval.

- **Ingesting PDF Documents**: We begin by loading PDF files and converting them to plain text, ensuring all content is accessible for downstream processing.

- **Chunking the Data**: After extracting the text, we split it into smaller chunks that fit within the maximum context window of our language model, allowing the model to process and retrieve relevant information effectively.

- **Creating Embeddings**: Using a Databricks-managed embedding model, we generate vector embeddings for each chunk, enabling fast and accurate retrieval.

- **Storing Embeddings in Delta Table**: The embeddings are then stored in a Delta table, providing a structured and efficient storage format. This setup ensures that the embeddings are easily accessible for querying and can scale with the needs of the model.

## Installation and Setup

- **Required Libraries**: Install the necessary libraries to ensure compatibility with the RAG model and data processing requirements.

- **Helper Functions**: Include essential helper functions for data manipulation and model interaction.

In [0]:
%pip install --quiet mlflow==2.14.3 transformers==4.30.2 llama-index==0.10.62 pydantic==2.8.2 accelerate==0.20.3 PyPDF2
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/_helper_functions


Extraction of raw data from the `Volumes` directory and storing it into a Delta table.

In [0]:
articles_path = f"/Volumes/workspace/default/raw_data"
table_name = f"pdf_raw_text"

# read pdf files
df = (
        spark.read.format("binaryfile")
        .option("recursiveFileLookup", "true")
        .load('/Volumes/workspace/default/raw_data')
        )

# save list of the files to table
df.write.mode("overwrite").saveAsTable(table_name)

display(df)

path modificationTime length content dbfs:/Volumes/workspace/default/raw_data/KB0000001.pdf 2024-10-23T06:04:16.000Z 6715 JVBERi0xLjcKJeLjz9MKNSAwIG9iago8PC9GaWx0ZXIvRmxhdGVEZWNvZGUvTGVuZ3RoIDE0NTk+PnN0cmVhbQp4nK1ZS3PbNhC+61fg6B6M4k3SMz3YcdODO44TK+NjRw/aZi1SiR7j0b/vAiRISsZKRKM4MzYGn75vd7nYXVA/R4xqxjNDDn9/+8tukfeRUCSRggpNypFOU79YjB5HX0c/B33esO7j9d/+05zYn9VLrZISkRgiFVnlo2e3fzMe/f6Zk5SMn0fCkMRwqjUZz0eCjFcgIIwxCYhYUpal5PC3syK8BaIXd9XyfZHPX3Lyd7He/Db+d/TneN8uyZgzjIs9u7p92DhluAafAXJu4x8mYDcPGQ3BtP+Px1JkVKvzWfNtW5GbHbkij7v1Ji/J9bwsKgjqarJZrrDIgpXHAnvCCSUV1Wd3RDChLjm7FKAvrpS54pw8TGbFczEjt5Pdonh53ZBxUeZI5E1mKOMk04RrmmUHposulxmWDsT+WCvdHy64+Y/lyqpuFvmV1b0gIXXOMy+vTBbUl7U+F7xnQCd0+kBYD1NGmTjtoUk11clADyF9biebnEyquYvuKTdrGwa42VnRqf3yMzaKMjUgAgZyMx0egenulNu18BC3W+lOYujRtB6qlDI9wEMtqc6GeTieTBc5qSanH24tPsTLVr6TeZv+8+bTGPMOehgzA7yTCTXDnPu6zVc78mlZzYtNsaxOeVgbMMRDb0Indb8tp/mK/EHubpj7hzUBw6EWJAPcFJwaPszPR1uIvqzm+eqUi7X4EBdb+Q8+FhWZ5+tZDlGtXsjSyvb0HJFqmqziNNFEZ5oasc/FyV0gHxhVUPftwOLqPgwl2ggKhVvQVBIhP5qsm4AlRCdgcajn1BGAgLWhuNiQQ4tNzSMNNTKK6h5hUoKaOKO2GFNiJ5UYphJh0opKHsU0xZgyyuLilCNMRtJERTGt8IThPLWJcippjM9z5tAR2o+IF1wYquLceMWoJKdcR1EtUSpN07gk/HCUWyqIbNwZw5igKIgsigo7rVxB8sRZNUepUqrirMIymkP/E3HHbI1SJVTFPcAZRgU3nV84aPtUKra2FSiVoUlcrH5gVMCh4k4zmqIJFA1xHgcT6LPmPKcZxmsTl+3Vkd7K6wY1rFQKHv3IrxE3BFTdyHaNdUYhFM3OU5WEiD5pWAEXMEfyuFKCPXIB80gc05HmKJmhpwcqPwixlMY1oU/YSAWpo+MKzwSlyuBjZ3neUtSj6RnqvbS9P86qF4zK9v64hMZSR9re//9zZ58Ken9cOdzhaagYo3LwjKaYcOgI7SdsjubsXIOVgpw+02CleBI5WL2hNw53f42hekaplH0NFUO1QKngSnWeoMPhiLy+vB9Jw0xRPrj/qcw4dIT2d+zqBBkdOR5iI4+Gih45p2Dzr+aO4xxVWnNDZVzyYFVa2/tcnFVYldbwdHVcIvZj1W7bk0F0mlCWkrJZwWjND74+sURgvC1dHuvXYbSdx2yGtczNOoyW9qB7qFsguCxzV9wW2qzDaJVJN+V5tF+H0RbWC0WzRLyDh5ga+/KzbC/p9ToQtqS3XbaDKgIXNnGTDt5MOQhaMt3tlm0vwtAQLdkzxdcMBG7DxXum2K/WULOVC5XpxcMt0XA04HZsD4J9MBqwn/iC2CYUDbTtymFsEwgP9rUzCPZhaMAuCgFk835PApXwBdjYcX6/AKvuBZ+G8n343cixt60+sAMl/OuggMzjZJGvyeflapaT6+1mWU7s++QP71pDbDA7mYN3xcWa3H55ug+Z7B/v0Kg017KAyU+TXZWTp3w6Dek0mTFQprkMBFSuq2q5rWZ5mVebdUjI59VAJT/uBaQettNFsX7NP1blXjcfKtO084CMYEJccnHJNBHMfg0lJd4G3PDgu4CUVhttAw3WLxF0W/Yb5maJoOvKX0Pd3xjOl/0G2iwRdFv2a7RfImhf92tws8K8U27LiDoQfhGKWQeEGKDA3l4XgRCwt+eKHQrs7blChwOznjOuyPWB/wG0HRA4CmVuZHN0cmVhbQplbmRvYmoKNCAwIG9iago8PC9Db250ZW50cyA1IDAgUi9NZWRpYUJveFswIDAgNjEyIDc5Ml0vUGFyZW50IDIgMCBSL1Jlc291cmNlczw8L0ZvbnQ8PC9GMSA2IDAgUi9GMiA3IDAgUi9GMyA4IDAgUi9GNCA5IDAgUi9GNSAxMCAwIFIvRjYgMTEgMCBSPj4+Pi9UcmltQm94WzAgMCA2MTIgNzkyXS9UeXBlL1BhZ2U+PgplbmRvYmoKMTMgMCBvYmoKPDwvRmlsdGVyL0ZsYXRlRGVjb2RlL0xlbmd0aCAyNTIxPj5zdHJlYW0KeJytWtuS2zYSfZ+v6EenVsPwfpl9Gsdx1ptsMhur1lVbeYEoSEKGJGRerGi/Jp+63QBBiZQEKlX2ZSxI3Th9+gpQ/vzgOpHrZTFM//31B/oIDg9+CEngO34E5UOUpmZRPHx8+PfD57v0Y/ekrl8bbQ/od73VKCn4SQxBCDV/2KjP3y4fvn3vQQrLzYMfQxJ7ThTBcv3gw7JGAD+O4wRBaFM3S2H6r7Li+kcI+ubHSh4Kvt5y+Ek07TfL3x++X47tClxXGeb5I7tOn+MHc4ZHyBlFvrbxLwzt9q8Zjc6kP3Zf+pkThV/Pml+7Ct4e4Qk+HpuWl/C8LkWFTq1ZK+tbnkUrbY6dIREGoRN9dSK+64ePnvvoI77/FMZPngcvLBcbkcM7dizEdtfCUpT8jJTrhLgHJb/aAxM8cSPH9SCKffASJ8smDPw+DJgV3o2s0J5AYweXnNL1wp8o5qMzIEGXlf1LNyQLxpWGclGaDIL96xuSVJN+4AyFr15flcINIs+I6cVlhUcKyqMPo1i9eRF0JRPjTzfByGdOghqhk53F/9sfGg+2jfJ5CuYvqRKTOEVV9HiQOF6AqXXp+GBwfJxmBIOOd4F+k4N/7soVr58G946M8+KYdp3HCDUGycfBVZwf37rql3cWSGTmW5glLmX5fczQZ1E6QRw63Yo1fIbhLNaE4RW8D8sxtcBCLYodN7qTGrZSNxtDfcdavpX1cYbULMqE1BWk56qSXZXzkldtM+YXWviFHrWp+/iFEbaREehLtypEs+PrGX6zKBN+l0iq87nRozdJy8jCDX9ict/HLfAc7KUjyP+wQqyhlTPcZlEm3K4g+W4QPLrxo+9da56ePqrE1HHiLNPdrF9SU0ou+55W8eKIZvugY9ZWpcANTxpqYRXHYR1nqYOTsBw8otc3jQqVG4wSQmhiNiVCSUPHT85Q9HoGxSgZFKsSocTY2LMzFL2eQTFKBsWqRCgRBuIMRC1nMHoVA2FTIYQQK8E7g9DrGQyjZECsSoSCWRj4Zyh6PYNilAzKNSV1BhtS3DZx46sNIHBTHe87Z27gZnT0vzYPn+tW5AWH9ri/NZvCMFGFde/0Jfn0+pT/x/JfP407XGIleO/oNQSvjMNPsn7dFPIwQ+7ewWvIXUEaxsWYYWpleO8ENgyvzMaPsqtzDkvWvM6QtIGhpZnV0ntnqbH0cso9ty3LdzTCoRDVnLH3TlUTkUu8DSsaPg6G51o53jtTDccr0+6daPYFOwIbyDYzRO8dsYboFdAp04upl55PvbkhGYZ6lhsNs7YqRdge/LMRbtYz4zjRSv24NC6xjssQI52mJ6UojWc0DIyZl0NHs81LA2OUFIxVw8CYgWlgrAPTwBglBWPVMDD9zByK2zIzDUivojBs8gbCjMyhJm0j04AYJYVi1TAwZmYOFWEbtAbGKCmYG1NW3WoDHN+Bvvn6EQLG4zut7VKLbQG7fIQJPHPAxo9jf9KWd7JuYc2bvBb7VsjqRhtQSFEWOt5NoHAAij


Let's view the content of one of the articles.

In [0]:
with open(f"{articles_path.replace('dbfs:','/dbfs/')}/KB0000029.pdf", mode="rb") as pdf:
  doc = parse_bytes_pypdf(pdf.read()) 
  print(doc)
  

Knowledge Details Page 1
Run By : System Administrator 2024-10-21 08:53:28 Pacific Daylight TimeReport Title: Knowledge Details
Run Date and Time: 2024-10-21 08:53:28 Pacific Daylight Time
Run by: System Administrator
Table name: kb_knowledge
Knowledge
Number: KB0000029
Knowledge base: IT
Category: Email
Published: 2014-09-09
Valid to: 2100-01-01Article type: HTML
Workflow: Published
Source Task:
Attachment link: false
Display attachments: false
Short description:
What is Spam?
Article body:
What is Spam? 
Spam has increasingly become a problem on the Internet. While every Internet user receives some spam, email addresses posted to web sites or in 
newsgroups and chat rooms attract the most spam.' 
Definitions 
The term "spam" is Internet slang that refers to unsolicited commercial email (UCE) or unsolicited bulk email (UBE). Some people refer to this kind of 
communication as junk email to equate it with the paper junk mail that comes through the US Mail. Unsolicited email most often 

To manage the text size and ensure it remains within the context window, we'll wrap the PDF text processing with a `SentenceSplitter`. This approach allows us to split large text into manageable chunks.

Additionally, we implemented a **Pandas UDF function** to scale this process across multiple nodes, enabling efficient parallel processing of the PDF text chunks.

In [0]:
import io
import os
import pandas as pd 
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document
from llama_index.core.utils import set_global_tokenizer
from transformers import LlamaTokenizerFast  # Updated import
from typing import Iterator
from pyspark.sql.functions import col, udf, length, pandas_udf, explode
from PyPDF2 import PdfReader  # Updated import

# Initialize LlamaTokenizerFast
tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

# Set LlamaTokenizerFast as the global tokenizer
set_global_tokenizer(tokenizer)

# Define a function to split the text content into chunks
@pandas_udf("array<string>")
def read_as_chunk(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    
    def extract_and_split(b):
        txt = parse_bytes_pypdf(b)  # Use custom function
        if txt is None:
            return []
        nodes = splitter.get_nodes_from_documents([Document(text=txt)])
        return [n.text for n in nodes]

    for x in batch_iter:
        yield x.apply(extract_and_split)

# Processing dataframe chunks
df_chunks = (df
                .withColumn("content", explode(read_as_chunk("content")))
                .selectExpr('path as pdf_name', 'content')
                )
display(df_chunks)


/local_disk0/.ephemeral_nfs/envs/pythonEnv-f9cf75fc-f339-4861-ac10-901c6ba025f3/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-f9cf75fc-f339-4861-ac10-901c6ba025f3/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pdf_name,content
dbfs:/Volumes/workspace/default/raw_data/KB0000001.pdf,"Knowledge List Page 1 Run By : System Administrator 2024-10-22 22:46:11 Pacific Daylight TimeReport Title: Knowledge List Run Date and Time: 2024-10-22 22:46:11 Pacific Daylight Time Run by: System Administrator Table name: kb_knowledge Query Condition: Number = KB0000001 Sort Order: Number in descending order 1 Knowledge ▼ Number Short description Author Category Workflow Updated KB0000001 Sales Force Automation is DOWNWayne Webb Announcements Published 2022-12-05 20:46:33 Knowledge List Page 2 Run By : System Administrator 2024-10-22 22:46:11 Pacific Daylight TimeKnowledge Number: KB0000001 Knowledge base: IT Category: Announcements Published: 2024-05-11 Valid to: 2033-06-21Article type: HTML Workflow: Published Source Task: Attachment link: false Display attachments: false Short description: Sales Force Automation is DOWN Article body: On Friday, January 20th, we experienced a widespread outage that affected all Zoho services. The outage started around 8:13 am Pacific Time. Zoho services started coming back online for customer use at 3:49 pm, and all services were fully restored at 6:22 pm PST. We absolutely realize how important our services are for businesses and users who rely on us; we let you down on Friday. Please accept our humblest apologies. The cause of the outage was an abrupt power failure in our state-of-the-art collocated data center facility (owned and operated by Equinix) in the Silicon Valley area, California. Equinix provides us physically secure space, highly redundant power and cooling. We get our internet connectivity from separate service providers. We own, maintain and operate the servers and the network equipment and the software. The problem was not just that the power failure happened, the problem was that it happened abruptly, with no warning whatsoever, and all our equipment went down all at once. Data centers, certainly this one, have triple, and even quadruple, redundancy in their power systems just to prevent such an abrupt power outage."
dbfs:/Volumes/workspace/default/raw_data/KB0000001.pdf,"Data centers, certainly this one, have triple, and even quadruple, redundancy in their power systems just to prevent such an abrupt power outage. The intent is that any power failure would have sufficient warning so that equipment, databases most importantly, can be shut down gracefully. In fact, the main function such data centers perform is to provide extreme redundancy in power systems, provide cooling for the equipment and provide physical security. Absolutely no warning happened prior to this incident, which is what we have asked our vendor to explain, and we hope they would be transparent with us. I do want to say that Equinix has served us well, they are a leader in this field, we have never suffered an abrupt power outage like this in 5+ years. But they do owe us and other customers in that data center an explanation for what happened on Friday. They restored power quickly, but the damage was done because of the abruptness of the outage. Wiki:"
dbfs:/Volumes/workspace/default/raw_data/KB0000002.pdf,"Knowledge List Page 1 Run By : System Administrator 2024-10-22 22:45:57 Pacific Daylight TimeReport Title: Knowledge List Run Date and Time: 2024-10-22 22:45:57 Pacific Daylight Time Run by: System Administrator Table name: kb_knowledge Query Condition: Number = KB0000002 Sort Order: Number in descending order 1 Knowledge ▼ Number Short description Author Category Workflow Updated KB0000002 Email Interruption Tonight at 11:00 PM Eastern Wayne Webb Announcements Published 2022-12-05 20:46:33 Knowledge List Page 2 Run By : System Administrator 2024-10-22 22:45:57 Pacific Daylight TimeKnowledge Number: KB0000002 Knowledge base: IT Category: Announcements Published: 2024-05-13 Valid to: 2033-06-21Article type: HTML Workflow: Published Source Task: Attachment link: false Display attachments: false Short description: Email Interruption

Lets try a hello world with our embedding model 'gte-base-en' model

In [0]:
from mlflow.deployments import get_deploy_client


# gte-base-en Foundation models api. 
deploy_client = get_deploy_client("databricks")
embeddings = deploy_client.predict(endpoint="RAGdatabricks-gte-base-en", inputs={"input": ["What is SPAM?"]})
print(embeddings)

{'id': '6582b26d-a70b-48a3-8d85-b0eefcebb2bc', 'object': 'list', 'model': '', 'data': [{'index': 0, 'object': 'embedding', 'embedding': [-0.259521484375, -0.474853515625, -0.73291015625, 1.2822265625, 0.59765625, -0.15234375, 0.9111328125, -0.351806640625, 0.73876953125, 0.51904296875, -0.61669921875, 0.411865234375, -1.650390625, 0.8935546875, 1.2490234375, 1.3037109375, 1.0263671875, 0.08984375, -1.4072265625, 0.0212860107421875, 0.20947265625, -1.9140625, 0.63232421875, -0.13427734375, 0.035919189453125, 1.568359375, 0.487548828125, 0.9150390625, 0.469970703125, -0.50830078125, 0.0703125, -0.1502685546875, 0.05242919921875, 0.267578125, 0.28369140625, 0.75048828125, -1.40625, 0.98828125, -0.2470703125, 0.99365234375, -0.90771484375, -1.51953125, -1.7490234375, -0.060211181640625, -0.5185546875, 0.3203125, -0.74658203125, -1.2373046875, 0.53759765625, 0.1834716796875, 0.1793212890625, 0.8623046875, 1.01953125, 0.501953125, 0.1248779296875, 0.258056640625, -0.492919921875, 0.325439453

### Compute Chunk Embeddings

The final step involves computing embeddings for all document chunks. We define a UDF to leverage the foundation model endpoint for this purpose.

In [0]:
@pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    def get_embeddings(batch):
        # NOTE: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="RAGdatabricks-gte-base-en", inputs={"input": batch})
        return [e["embedding"] for e in response.data]

    # splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)

In [0]:
import pyspark.sql.functions as F

df_chunk_emd = (df_chunks
                .withColumn("embedding", get_embedding("content"))
                .selectExpr("pdf_name", "content", "embedding")
                )
display(df_chunk_emd)

pdf_name content embedding dbfs:/Volumes/workspace/default/raw_data/KB0000001.pdf Knowledge List Page 1
Run By : System Administrator 2024-10-22 22:46:11 Pacific Daylight TimeReport Title: Knowledge List
Run Date and Time: 2024-10-22 22:46:11 Pacific Daylight Time
Run by: System Administrator
Table name: kb_knowledge
Query Condition: Number = KB0000001
Sort Order: Number in descending order
1 Knowledge
▼ Number Short description Author Category Workflow Updated
KB0000001 Sales Force Automation 
is DOWNWayne Webb Announcements Published 2022-12-05 20:46:33
Knowledge List Page 2
Run By : System Administrator 2024-10-22 22:46:11 Pacific Daylight TimeKnowledge
Number: KB0000001
Knowledge base: IT
Category: Announcements
Published: 2024-05-11
Valid to: 2033-06-21Article type: HTML
Workflow: Published
Source Task:
Attachment link: false
Display attachments: false
Short description:
Sales Force Automation is DOWN
Article body:
  
On Friday, January 20th, we experienced a widespread outage that affected all Zoho services. The outage started around 8:13 am Pacific Time. Zoho 
services started coming back online for customer use at 3:49 pm, and all services were fully restored at 6:22 pm PST. We absolutely realize how important 
our services are for businesses and users who rely on us; we let you down on Friday. Please accept our humblest apologies.  
  
The cause of the outage was an abrupt power failure in our state-of-the-art collocated data center facility (owned and operated by Equinix) in the Silicon 
Valley area, California. Equinix provides us physically secure space, highly redundant power and cooling. We get our internet connectivity from separate 
service providers. We own, maintain and operate the servers and the network equipment and the software. The problem was not just that the power failure 
happened, the problem was that it happened abruptly, with no warning whatsoever, and all our equipment went down all at once. Data centers, certainly this 
one, have triple, and even quadruple, redundancy in their power systems just to prevent such an abrupt power outage. List(0.004798889, -0.2121582, -0.91552734, -0.5805664, 0.5415039, -0.07086182, 1.4541016, -0.40161133, 1.4707031, -0.20153809, 1.0009766, 0.33911133, 0.96240234, 1.0800781, -0.85009766, 0.34814453, 0.9682617, 0.077941895, 0.009094238, -0.18774414, 1.1962891, -1.9072266, 0.22094727, 0.07873535, -0.16723633, 0.95166016, -0.30126953, -0.041748047, -0.6591797, -1.5810547, -1.8476562, -0.029067993, 0.87939453, 0.28125, 0.6635742, -0.72802734, -0.74365234, -1.03125, 1.0283203, -0.07739258, -1.3125, 0.99853516, -0.017532349, 1.4394531, -0.55615234, 1.7382812, -1.0498047, -1.1210938, 0.6230469, 1.6162109, 0.40429688, 1.0078125, -0.5517578, 0.484375, -0.78027344, -0.21411133, 0.5883789, -0.30444336, -0.021392822, -0.7792969, -0.08282471, 0.9116211, 0.48339844, -0.11462402, 0.33764648, -0.37475586, 0.56347656, -0.078552246, -0.25512695, 0.05545044, 0.9057617, -0.42993164, 1.1503906, -1.6259766, -1.1298828, 0.32617188, -0.5205078, 0.43920898, -0.421875, 1.3564453, -0.12841797, 0.14111328, -0.67041016, -1.015625, -0.43774414, -0.107055664, -0.5756836, -0.5385742, -1.0361328, -0.7104492, -0.31713867, -0.5161133, -0.01739502, 0.69970703, 1.0097656, -0.7734375, 0.6894531, 0.55566406, -0.07092285, 0.87060547, -0.44580078, 1.40625, 0.77001953, 0.61328125, -0.59228516, -0.9477539, 0.50097656, -0.66503906, -0.41308594, 0.80859375, 0.0501709, 1.0810547, -1.2929688, -0.013153076, -1.1455078, 0.71972656, -1.0039062, -0.21899414, 0.42504883, 0.375, -0.8666992, 0.49536133, -0.85498047, -0.4506836, 0.8574219, -0.23535156, 0.80566406, 0.16711426, -0.8330078, 0.32104492, 1.2294922, 0.21472168, -0.66064453, -0.64697266, -0.49829102, 0.89208984, -0.041931152, -0.70751953, -0.14855957, -0.19494629, 1.7636719, 0.05822754, -0.6870117, 0.2763672, 0.009338379, 0.6879883, -0.25195312, -0.5083008, -1.0419922, 0.18164062, 0.70214844, 0.6425781, 0.03741455, -0.28588867, -0.11993408, -0.7832031, -0.4775

Save the embeddings into a Delta Live Table.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS pdf_text_embeddings (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  pdf_name STRING,
  content STRING,
  embedding ARRAY <FLOAT>
  -- NOTE: the table has to be CDC because VectorSearch is using DLT that is requiring CDC state
  ) TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
embedding_table_name = "pdf_text_embeddings"
df_chunk_emd.write.mode("append").saveAsTable(embedding_table_name)